In [2]:
import os, re
from os.path import join
import pandas as pd
import numpy as np

### 전체계좌 / 일부계좌 구분

In [6]:
# 파일 읽기
####################################################################
path_account = r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847" + ".pkl" # 계좌전체. 채무상태나 담당자 변경하려면 최신파일로
path_work_df = r'c:/Users/SL/Desktop/새 스프레드시트 문서.xlsx' # 확인하려는 풀의 채무자키와 계좌키
path_save = r'c:/Users/SL/Desktop/전체계좌여부.xlsx'
####################################################################
all_df = pd.read_pickle(path_account)[["채무자키","계좌키","종결일","채무상태","담당자","채무자명"]].fillna("")
work_df = pd.read_excel(path_work_df, dtype={"채무자키":str,"계좌키":str})
print(len(all_df), len(work_df))

78158 453


In [7]:
# 채무자키로 전 계좌 불러오기
work_all_account = all_df[all_df.채무자키.isin(work_df.채무자키)].copy() # 객체 새로 만들지 않으면 계속 copywarning 나옴
# 전 계좌 중 작업 계좌에 있거나, 종결된 계좌면 Y, 아니면 N(솔림 잔존계좌)
cond = (work_all_account.계좌키.isin(work_df.계좌키)) | (work_all_account.종결일 != "")
work_all_account["계좌작업대상여부"] = np.where(cond, "Y", "N")

# 채무자키별로 전계좌 여부 확인 후 파일 저장하기
result = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = work_all_account.drop_duplicates('채무자키').copy()
result["채무자키"] = temp.채무자키.values
result["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result["담당자"] = temp.담당자.values
result["채무자명"] = temp.채무자명.values


# 작업대상여부의 유니크값이 1이면 모두 Y (모두 N인 경우는 논리적으로 불가능), 2면 작업계좌와 잔존계좌가 혼재한 것이므로 일부계좌
g = work_all_account.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result.iterrows() : 
     if g.get_group(v.채무자키).계좌작업대상여부.nunique() == 1 :
          result.loc[i, "전계좌여부"]="전계좌"
     else : result.loc[i, "전계좌여부"]="일부계좌"

# 저장하기
result.to_excel(path_save, index=False)